**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
df=pd.read_csv("cleaned_data.csv")
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26659,ecff42d0-bdc6,220000,1,3,1,39,0,0,0,0,...,5003,3047,5000,1000,0,high school,0,1,0,0
26660,99d1fa0e-222b,150000,1,3,2,43,-1,-1,-1,-1,...,8998,129,0,0,0,high school,0,1,0,0
26661,95cdd3e7-4f24,30000,1,2,2,37,4,3,2,-1,...,22000,4200,2000,3100,1,university,0,0,0,1
26662,00d03f02-04cd,80000,1,3,1,41,1,-1,0,0,...,1178,1926,52964,1804,1,high school,0,1,0,0


In [4]:
# Define the sigmoid function
def sigmoid(x):
  return 1 / (1 + np.exp(-(x)))

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [5]:
# Create a train/test split
X= df[['LIMIT_BAL','PAY_1']]
y=df['default payment next month']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='liblinear')
y_test.shape

(8000,)

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [7]:
# Fit the logistic regression model on training data
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
# Make predictions using `.predict()`
y_pred = classifier.predict(X_test)
print(np.unique(y_pred,return_counts=True))

from sklearn.metrics import accuracy_score , confusion_matrix 


result2 = accuracy_score(y_test,y_pred)
print("Accuracy of the model==",result2*100,'%')

a=confusion_matrix(y_test,y_pred)
print("\nConfusion Matrix==\n",a,'\n')

(array([0, 1], dtype=int64), array([7357,  643], dtype=int64))
Accuracy of the model== 81.39999999999999 %

Confusion Matrix==
 [[6063  194]
 [1294  449]] 



In [9]:
# Find class probabilities using `.predict_proba()`
y_pred_proba = classifier.predict_proba(X_test)
y_pred_proba

array([[0.49719554, 0.50280446],
       [0.6245546 , 0.3754454 ],
       [0.89790065, 0.10209935],
       ...,
       [0.91237662, 0.08762338],
       [0.87776599, 0.12223401],
       [0.86647072, 0.13352928]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [10]:
# Add column of 1s to features
features=y_pred_proba[:,1]
features

array([0.50280446, 0.3754454 , 0.10209935, ..., 0.08762338, 0.12223401,
       0.13352928])

In [11]:
# Get coefficients and intercepts from trained model
theta_1 = classifier.coef_[0][0]
theta_2 = classifier.coef_[0][1]
print('Theta 1:--',theta_1,'\nTheta 2:--',theta_2)

Theta 1:-- -0.21978407194458333 
Theta 2:-- 0.7922131557668461


In [17]:
# Manually calculate predicted probabilities
a=sigmoid(-0.21978407194458333+0.7922131557668461*X_test)
a

array([[0.43405257, 0.76852624],
       [0.2819145 , 0.62106792],
       [0.49454354, 0.2854133 ],
       ...,
       [0.64266396, 0.2854133 ],
       [0.32030877, 0.2854133 ],
       [0.24645368, 0.2854133 ]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [29]:
# Manually calculate predicted classes

predicted_class=a[:,1]
aa=[]
for i in predicted_class:
    if i>=0.5:
        aa.append(1)
    else:
        aa.append(0)
aa=np.asarray(aa)
print('Manual method:--',np.unique(aa,return_counts=True))

print("\n\nSckit's learn:--",np.unique(y_pred,return_counts=True))

print('\n\nActual:--',np.unique(y_test,return_counts=True))


Manual method:-- (array([0, 1]), array([6209, 1791], dtype=int64))


Sckit's learn:-- (array([0, 1], dtype=int64), array([7357,  643], dtype=int64))


Actual:-- (array([0, 1], dtype=int64), array([6257, 1743], dtype=int64))


In [24]:
# Compare to scikit-learn's predicted classes
result1 = accuracy_score(y_test,aa)
print("Accuracy of the model using manual method==",result1*100,'%')



y_pred = classifier.predict(X_test)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy of the model using scikit-learn inbuilt function==",result2*100,'%')


Accuracy of the model using manual method== 77.9 %
Accuracy of the model using scikit-learn inbuilt function== 81.39999999999999 %


______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [20]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import  roc_auc_score

ras1=roc_auc_score(y_test,y_pred)
print("ROC AUC score of scikit-learn:--",ras1)

ROC AUC score of scikit-learn:-- 0.6132982809110366


In [22]:
# Use manually calculated predicted probabilities to calculate ROC AUC
ras2=roc_auc_score(y_test,aa)
print("ROC AUC score of manual method:--",ras2)

ROC AUC score of manual method:-- 0.6807401757077397
